<h1> The Problem </h1>

We have an Asana project which have some information about Procurement Requests

The problem was that we´ve Custom Fields and some information extracted via Asana forms that comes in Notes Field of a Task

I needed to create an ETL script that join all information in a csv file. This CSV was a source of a PowerBI Dashboard

At the moment that I´m writting this tutorial, Asana has problems when we try to export the project data to CSV cause we can split the custom fields and notes fields with excel or something like this. So, I needed to use the Asana API

<h1> Requirements </h1>

I used Asana API to extract the info. First of All, you need to have an Asana Account and, then, configure your PAT (Personal Access Token)

https://developers.asana.com/docs/personal-access-token#:~:text=A%20personal%20access%20token%20(PAT,not%20unlimited%2C%20personal%20access%20tokens.

<h1> The logic behin the ETL </h1>

I used 3 layers in order to extract the information. This approach gave me some flexibility in order to change things a long the way.

My reference is Data Engineering topics like this: https://www.databricks.com/glossary/medallion-architecture

1) Bronze Layer

2) Silver Layer

3) Golden Layer

With the result of the Bronze Layer, you´ll be able to create the Silver Layer and, then, with the result of the Silver Layer you´ll be able to create the golden layer

<h1> How to use it </h1>


You´ll neeed

1) Put all the fields you want to extract in a list (including the notes fields) like 
    list_of_fields = ['aaa', 'bbb']

2) Run extract_bronze_layer function passing:
    Asana Project ID
    'prod or test' ->
    list of fields that It was mentioned in step 1

    * filtered_bronze_layer = {task: task_fields for task, task_fields in bronze_layer.items() if task_fields[2] != 'Unknow'}

3) Run extract_silver_layer passing the filtered_bronze_layer

4) Run extract_golden_layer passing
    silver_layer extracted before
    Asana Project ID
    

<h1><font color='blue'> Please, put the information here </h1></font>

In [ ]:
personal_access_token = ''
asana_project_id = ''
field_list = ['', '', '']

<h1> Bruno´s way of code </h1>

I´ve been using functional programming and I believe that I can use some background concepts in order to give me a concise code. Python is not a functional language, but it has the power of using some functional concepts. 

Sometimes it´s better to use Object Oriented approach like using Dataclasses, but, in this specific problem, I didn´t use it.

1) Function instead of Objects

2) High-order functions

3) Immutability driven

4) Create datastructures combining Dictionaries, Lists and Tuples

5) Iterate over datastructures using List and Dict Comprehension

<h1> IMPORTS </h1>

In [15]:
# Libs that we´ll use during the code #

import asana
import pandas as pd
import re
from datetime import datetime, date
######################################


#### Asana Parameters ####
client = asana.Client.access_token(personal_access_token)
client.LOG_ASANA_CHANGE_WARNINGS = False #Disable Asana API Warnings
headers = {
    'Accept': 'application/json',
    'Authorization': 'Bearer {personal_acess_token}',
}
##########################



#### Date Parameters ####
hoje = datetime.today()
today = datetime.today()
dia = hoje.strftime("%Y-%m-%d")
date_today = date(hoje.year, hoje.month, hoje.day)
#########################

<h1> BRONZE LAYER Functions </h1>

As you can see, I´m using Python Closure in join some functions into a single one

Reference:
https://www.programiz.com/python-programming/closure#:~:text=Python%20closure%20is%20a%20nested,of%20nested%20functions%20in%20Python.

In order to understand the logic i´ve splitted the problem into 3 steps:

1) Find the regular values of a task (name, created_at, modified_at, assignee) - All tasks, at minimum, will have this fields. See the function <i>get_default_fields()</i> Be careful with assignee, because sometimes we can retrive "None" as the value.

2) Find the custom fields of a task. See the function <i>try_get_regular_or_custom_field()</i>

3) Extract the Notes Fields. See the function <i>try_get_notes_fields()</i>

Finally, I aggregate them into a dictionary that will be the Bronze Layer

IMPORTANT: There are some cases that you´ll not be able to extract the notes fields of a task. So, I classified it as "unknow". You´ll need to filter this before create the Silver Layer

In [4]:
def extract_bronze_layer(project_gid: str, environment, fields_list: list):
    
    def choose_right_iterator(prod_or_test: str):
        
        """This is a way to avoid wasting time debugging. If you choose test, the script will read only the first 10 entries"""
        
        iterator_for_debug_meaning = list(client.tasks.get_tasks_for_project(project_gid))[0:10]
    
        iterator_with_task_list_of_a_project = client.tasks.get_tasks_for_project(project_gid)

        return iterator_for_debug_meaning if prod_or_test == 'test' else iterator_with_task_list_of_a_project
    
    def try_regular_AND_non_regular_fields(task_gid: str, field_name_list: list):
        
        """I called the method to get task information and store all the values into task_infos_generator"""
        
        task_infos_generator = client.tasks.get_task(task_gid)
        
        def get_default_fields(task_infos_generator):
            
            return {'name': task_infos_generator['name'],
                    'created_at': task_infos_generator['created_at'][:10],
                    'modified_at': task_infos_generator['modified_at'][:10],
                    'assignee': task_infos_generator['assignee']['name'] if task_infos_generator['assignee'] != None else None} 
        
        
        def try_get_regular_or_custom_field(task_infos_generator, field_name_list: list):
        
            def get_regular_or_custom_field(task_infos_generator, field_name: str):
                
                try:
                    field_value = task_infos_generator[field_name] 
                except:
                    try:
                        field_value = [task['display_value'] for task in task_infos_generator['custom_fields'] if task['name'] == field_name][0] 
                    except:
                        field_value = 'NA'
                
                return field_value
            
            return {field: get_regular_or_custom_field(task_infos_generator, field) for field in field_name_list}
        
        
        def try_get_notes_fields(task_infos_generator):
            
            """I needed to extract the notes fields and put them into a tuple"""
            
            def extract_notes_from_field(task_infos_generator):
            
                string = task_infos_generator['notes']

                string_content = re.compile("\n").split(string)
                ##### You´ll see that the information were spplited by "\n" in notes fields. So, I used this to create a list with the values ####
                
                return string_content

            def create_tuple_with_fields_values(list_of_string_values):

                    def create_end_interval_list(list_of_string_values: list):
                        
                        string_values = list_of_string_values.copy()
                            
                        end_interval_list = []

                        for index in list(range(0, len(string_values))):
                            if string_values[index] == "":
                                end_interval_list.append(index)
                            
                        return end_interval_list
                        
                    def create_start_interval_list(end_interval_list: list):
                        
                        start_interval_list = end_interval_list.copy()
                            
                        del start_interval_list[-1] 
                        start_interval_list = [element + 1 for element in start_interval_list]
                        start_interval_list.append(0)
                        start_interval_list.sort()
                            
                        return start_interval_list
                                
                    def create_iterator_with_list_slice(list_of_string_values, end_interval_list, start_interval_list):
                            
                        string_values_list = list_of_string_values.copy()
                            
                        for start, end in zip(start_interval_list, end_interval_list):
                            yield string_values_list[slice(start, end)]    
                        
                    def create_dictionary_with_fields_values(adjusted_list):
                            
                        fields_name_list = []
                        for content in adjusted_list:
                            try:
                                field_name_content = content[0]
                            except:
                                field_name_content = 'Unknown'
                            fields_name_list.append(field_name_content)
                            
                        fields_value_list = []
                        for content in adjusted_list:
                            try:
                                field_value_content = " ".join(content[1:])
                            except:
                                field_value_content = 'Unknown' 
                            fields_value_list.append(field_value_content)
                            
                        dict_with_fields_values = {field: value for field, value in zip(fields_name_list, fields_value_list)}
                            
                        return dict_with_fields_values
                        
                    end_range = create_end_interval_list(list_of_string_values) 
                    start_range = create_start_interval_list(end_range)
                    adjusted_list_of_string_values = list(create_iterator_with_list_slice(list_of_string_values, end_range, start_range))
                    dict_with_fields_values = create_dictionary_with_fields_values(adjusted_list_of_string_values)
                        
                    info = dict_with_fields_values
                        
                    return info
                
            return create_tuple_with_fields_values(extract_notes_from_field(task_infos_generator))
        
        
        default_fields = get_default_fields(task_infos_generator)
        custom_regular_fields = try_get_regular_or_custom_field(task_infos_generator, field_name_list) 
        
        try: 
            note_fields = try_get_notes_fields(task_infos_generator) 
        except:
            note_fields = 'Unknow'
        
        
        return (default_fields, custom_regular_fields,
                note_fields)
            
            
    task_list_generator = (task for task in choose_right_iterator(environment))
    
    task_list = [task['gid'] for task in task_list_generator]
    
    
    return {task: try_regular_AND_non_regular_fields(task, fields_list) for task in task_list}



     

<h1> Understanding the Bronze Layer </h1>

The final result will be a dictionary that will have the keys as the Asana Task Id´s. The value will be a tuple containing 3 dictionaries as explained in the image.

<img src='updated_bronze_layer.jpg'>

In [9]:
def extract_silver_layer(filtered_bronze_dict: dict):

    
    def filter_bronze_layer_with_default_fields(filtered_bronze_dict: dict):
       return {task: {field: field_value for field, field_value in task_fields[0].items() if field_value != 'NA'} for task, task_fields in filtered_bronze_dict.items()} 
    
    def filter_bronze_layer_with_regular_fields(filtered_bronze_dict: dict):
        return {task: {field: field_value for field, field_value in task_fields[1].items() if field_value != 'NA'} for task, task_fields in filtered_bronze_dict.items()} 

    def filter_non_regular_fields(filtered_bronze_dict: dict):
        return {task: {field: field_value for field, field_value in task_fields[2].items() if field_value != 'NA'} for task, task_fields in filtered_bronze_dict.items()}

    def filter_NA_tasks_fields(filtered_bronze_dict: dict):
        return {task: [field for field, field_value in task_fields[1].items() if field_value == 'NA'] for task, task_fields in filtered_bronze_dict.items()}

    def extract_NA_tasks_fields_in_bronze_layer(NA_task, non_regular_bronze_dict: dict):
        
        def get_field_value(filtered_dict: dict, field):
            try:
                return filtered_dict[field]
            except:
                return None
        
        return {task: {field: get_field_value(non_regular_bronze_dict[task], field) for field in task_field} for task, task_field in NA_task.items()}


    filtered_bronze_layer_with_default_fields = filter_bronze_layer_with_default_fields(filtered_bronze_dict)
    
    filtered_bronze_layer_with_regular_fields = filter_bronze_layer_with_regular_fields(filtered_bronze_dict)
    
    filtered_bronze_layer_with_non_regular_fields = filter_non_regular_fields(filtered_bronze_dict)
    
    filtered_NA_tasks_fields = filter_NA_tasks_fields(filtered_bronze_dict)
    
    extracted_NA_tasks_fields_in_bronze_layer = extract_NA_tasks_fields_in_bronze_layer(filtered_NA_tasks_fields,
                                            filtered_bronze_layer_with_non_regular_fields) 
     

    silver_layer = {
    
            task: 
        
        {
        
        **{field: fields_values #I used this ** in order to join 3 dictionaries into single one
            for field, fields_values in filtered_bronze_layer_with_default_fields[task].items()},
        
        **{field: fields_values
            for field, fields_values in filtered_bronze_layer_with_regular_fields[task].items()},
        
        **{field: fields_values
            for field, fields_values in extracted_NA_tasks_fields_in_bronze_layer[task].items()}
        
        }
    
        for task in list(filtered_bronze_layer_with_default_fields.keys())
       
       
        }
    
    return silver_layer
             


<h1> Understanding the Silver Layer </h1>

The final result will be a dictionary that will have the keys as the Asana Task Id´s. The value will be a tuple containing 3 dictionaries as explained in the image (as we had on the Bronze Layer)

But, it´s filtered with only the fields associated with the tasks. (We´ve removed the NA values)

<img src='silver_layer.jpg'>

In [10]:
def extract_golden_layer(silver_layer_dict: dict, project_gid: str):
    
    project_followers_list = client.projects.get_project(project_gid)['followers']
    
    dict_base_for_sections = dict_base = {section['name']: [task['gid'] for task in client.tasks.get_tasks_for_section(section['gid'])]
        for section in client.sections.get_sections_for_project(project_gid)}
    
    
    def extract_section_and_owner_email_from_task(task_gid, task_assignee, project_followers_list):
    
        
        extract_section_of_a_given_task = lambda task_gid: [key for key, value_list in dict_base_for_sections.items() for value in value_list if value == task_gid][0]
        
            
        def extract_owner_email_from_task(task_assignee, project_followers_list): 

            def find_e_mail_from_user_name(name, project_followers_list):
                    
                if task_assignee == None:
                    return None
                else:
                    try:
                        dict_with_name_gid = {name: gid for name, gid in zip([info['name'] for info in project_followers_list], [info['gid'] for info in project_followers_list])}
                        return client.users.get_user(dict_with_name_gid[name])['email']
                    except:
                        return None
                
            return find_e_mail_from_user_name(task_assignee, project_followers_list)

        
        return {'assignee_email': extract_owner_email_from_task(task_assignee, project_followers_list),
            'section': extract_section_of_a_given_task(task_gid)}
            
    
    task_list = list(silver_layer_dict.keys())
    
    
    layer_with_section_email = {task: extract_section_and_owner_email_from_task(task, silver_layer_dict[task]['assignee'], project_followers_list) for task in task_list}
    
    golden_layer = {task: {**{field: fields_values
            for field, fields_values in layer_with_section_email[task].items()},
        
        **{field: fields_values
            for field, fields_values in silver_layer_dict[task].items()}
        } for task, values in silver_layer_dict.items()}
    
    return golden_layer

<h1> Understanding the Golden Layer </h1>

The logic is similiar as Bronze Layer and Silver Layer, but I used this last layer in order to run the most deep processes. As you saw in the code, we had a function called <i>extract_section_and_owner_email_from_task()</i>

<img src='golden_layer.jpg'>

<h2><font color='green'> Bronze Layer </h2></font>

In [18]:
bronze_layer = extract_bronze_layer(asana_project_id, 'prod', field_list)

filtered_bronze_layer = {task: task_fields for task, task_fields in bronze_layer.items() if task_fields[2] != 'Unknow'}

<h2><font color='green'> Silver Layer </h2></font>

In [19]:
silver_layer = extract_silver_layer(filtered_bronze_layer)

<h2><font color='green'> Golden Layer </h2></font>

In [20]:
golden_layer = extract_golden_layer(silver_layer, asana_project_id) 

In [21]:
df = pd.DataFrame.from_records([value for value in golden_layer.values()], index=list(golden_layer.keys()))

<h1> Understanding what should we do with the Golden Layer </h1>

After ingest and processing all data through the Bronze, Silver and Golden Layer. I decided to transform all the data into a Pandas DataFrame in order to export it to an CSV file.

Remember that this data will be a source of a PowerBI Dashboard

<img src='dataframe.jpg'>

In [160]:
df.to_csv('ETL_extracted.csv')